In [125]:
%load_ext jupyternotify
import numpy as np
import pandas as pd
import glob
import pickle

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [126]:
# Pickling functions used to save and load the dictionary file

def save_obj(obj, name):
    with open('dataset/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('dataset/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [447]:
# Importing UN data
data=[]
name=[]
for file_name in glob.glob('dataset/UNDP_HDI/*.csv'):
    data.append(pd.read_csv(file_name,
                             skiprows=0,
                             na_values="..",
                             na_filter=True,
                             header=1))
    name.append(file_name.split('/')[-1].split('.')[0])

# This snippet is for dealing with the format of the imported dataframes
for i in range(len(data)):
    for col in data[i].columns:
        # Drop columns containing 'Unnamed'
        if "Unnamed" in col: data[i].drop(labels=col, axis=1, inplace=True) 

In [645]:
data_train = pd.read_csv('dataset/train.csv')
columns_train = list(data_train.columns)

In [133]:
# WARNING!------------------------------------------------------------------------
# This code snippet is not to be run if one wants to get a map
# of columns. In that case refer to the *pickled dictionary file* in the dataset
# folder. This is due to the manual corrections present later in this block, which
# are dependant on a certain sequence of columns from UN data, which in turn
# depends on the glob order. This load order might change on your run!

# This code is kept only to record my steps. 

import re

# Function str_comp compares how similar two strings are
# based on matching words or numbers.
# It returns a value from 0 to 1 (if duplicate words present could be more than 1).
# 0 means strings lstr1 and lstr2 contain completely different words
# and the larger the metric — the better similarity. 
# It's very crude, but gets the job done relatively well for my purpose. 

def str_comp(lstr1, lstr2): # lstr1 and lstr2 are strings to be compared
    
    # Forming 2 lists of words from strings, 
    # ignoring all non alphabet/number symbols.
    # filter removes unnecessary blank strings in the list
    # left by re.split.
    l1 = list(filter(None, re.split(r'\W',lstr1)))
    l2 = list(filter(None, re.split(r'\W',lstr2)))
    
    # matchings DO account for duplicate words in one or both lists of strings,
    # in which case it inflates the final metric to more than 1.
    # Ideally it should be fixed, but it works fine for comparing strings. 
    matchings = 0
    for s1 in l1:
        for s2 in l2:
            if s1.lower() == s2.lower(): matchings+=1
    return matchings/max([len(l1), len(l2)])


# Making a list containing indices to match columns between datasets
# based on the str_comp function 'metric'
col_map=[]
for i, col1 in enumerate(name):
    maxim = 0
    j_m = 0
    for j, col2 in enumerate(columns_train):
        comp = str_comp(col1,col2)
        # This part checks whether this combination of list of strings col1 and col2
        # are more similar than the previous best in this loop. It also checks
        # that the second list of strings col2 has not appeared before in
        # col_map to avoid duplicates. 
        if (comp >= maxim) and (j not in [x[1] for x in col_map]): 
            maxim = comp
            j_m = j
    col_map.append((i, j_m))

# Manually fixing mistakes made by the mapping script
# NOTE! This manual corrections may change on different loads of the data
#
# Please, use the columns dict pickle file in the dataset folder 
# for final mapping. 

corrections = {
    (37, 72): (37, 8),
    (43, 78): (43, 72),
    (56, 41): (56, 42),
    (57, 8): (57, 78),
    (10, 42): (10,41),
}

# Final list with corrections:
col_map_corr = [corrections.get(x,x) for x in col_map]

# Now it's nice and dandy
for i, j in col_map_corr:
    print(i, name[i])
    print(j, columns_train[j], '\n')

0 Education Index
9 Education Index 

1 Employment to population ratio (% ages 15 and older)
34 Employment to population ratio (% ages 15 and older) 

2 Population, ages 65 and older (millions)
20 Population, ages 65 and older (millions) 

3 Mortality rate, infant (per 1,000 live births)
17 Mortality rate, infant (per 1,000 live births) 

4 Human Development Index (HDI), female
70 Intergalactic Development Index (IDI), female 

5 Estimated gross national income per capita, female (2011 PPP$)
56 Estimated gross galactic income per capita, female 

6 Share of seats in parliament (% held by women)
47 Share of seats in senate (% held by female) 

7 Labour force participation rate (% ages 15 and older), male
33 Labour force participation rate (% ages 15 and older), male 

8 Remittances, inflows (% of GDP)
62 Remittances, inflows (% of GGP) 

9 International inbound tourists (thousands)
64 Intergalactic inbound tourists (thousands) 

10 Infants lacking immunization, measles (% of one-year-ol


This code was used to save the pickled dict.  
```python
col_map_abs = {}
for i, j in col_map_corr:
    col_map_abs[name[i].strip()] = columns_train[j].strip()

save_obj(col_map_abs, 'column_dict_from_UN_to_Contest_format')
```

In [549]:
# Forming a complete list of all stripped entries in the Country column

full_list_dirty = set()
for i in range(len(data)):
    countries = [x for x in data[i].loc[:, 'Country'] if str(x) != 'nan']
    countries_stripped = set(map(lambda x: x.strip() , countries))
    full_list_dirty |= countries_stripped

In [569]:
entries_to_remove = set([
    'Arab States',
    'Developing Countries',
    'High human development',
    'Human Development',
    'Least Developed Countries',
    'Low human development',
    'Medium human development',
    'Organization for Economic Co-operation and Development',
    'Small Island Developing States',
    'South Asia',
    'Very high human development',
    'World',
    'East Asia and the Pacific',
    'Latin America and the Caribbean',
    'Regions',
    'Sub-Saharan Africa',
    
])
full_list = full_list_dirty - entries_to_remove
full_list = sorted(list(full_list))

In [323]:
# Index rows of each dataframe in data by Country column
for i in range(len(data)):
    data[i].loc(:, 'Country')
    data[i].set_index('Country', inplace=True)

In [325]:
# Select only rows with countries and columns with years and the country list
for i,df in enumerate(data):
    data[i] = df.iloc[0:189, 1:]

In [583]:
for i in range(len(data)):
    for j, country in enumerate(data[i].loc[:, 'Country']):
        if str(country) != 'nan':
            data[i].loc[j, 'Country'] = country.strip()

In [403]:
i=37
print(name[i])
d = data[i].iloc[0:189, 1:]
d

Human Development Index (HDI)


,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.304,0.312,0.308,0.303,0.327,0.331,0.335,0.339,0.343,0.345,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
Albania,0.625,0.608,0.611,0.617,0.629,0.639,0.639,0.649,0.660,0.667,...,0.729,0.740,0.759,0.771,0.781,0.787,0.788,0.788,0.789,0.791
Algeria,0.582,0.589,0.593,0.597,0.602,0.610,0.619,0.629,0.638,0.646,...,0.720,0.730,0.738,0.737,0.746,0.749,0.751,0.755,0.758,0.759
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.759,...,0.830,0.828,0.827,0.849,0.846,0.853,0.850,0.854,0.852,0.857
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.384,0.394,...,0.508,0.510,0.525,0.537,0.547,0.557,0.565,0.570,0.576,0.574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela (Bolivarian Republic of),0.648,0.654,0.656,0.657,0.660,0.662,0.666,0.668,0.670,0.672,...,0.752,0.753,0.764,0.767,0.772,0.770,0.763,0.752,0.735,0.726
Viet Nam,0.484,0.496,0.506,0.517,0.529,0.540,0.539,0.559,0.566,0.578,...,0.650,0.653,0.663,0.668,0.673,0.675,0.680,0.685,0.690,0.693
Yemen,0.396,0.395,0.398,0.398,0.393,0.408,0.418,0.430,0.423,0.432,...,0.503,0.499,0.511,0.501,0.506,0.504,0.493,0.477,0.463,0.463


In [246]:
cols = [int(x) for x in d.columns[1:]]

In [571]:
idx = pd.IndexSlice

undp_ind = pd.MultiIndex.from_product([cols, full_list], names=['year', 'country'])
undp_data = pd.DataFrame(None, index = undp_ind)

In [591]:
data[0][data[0]['Country'] == 'Afghanistan'].loc[:, '1990']

0    0.122
Name: 1990, dtype: object

In [608]:
np.data[0][data[0]['Country'] == 'a'].loc[:, '1990'].values[0]

numpy.ndarray

In [597]:
data[0].columns[2:]

Index(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018'],
      dtype='object')

In [613]:
a = []
len(a) == 0

True

In [619]:
# To be commented 

for df, nam in zip(data, name):
    for country in full_list:
        for year in df.columns[2:]:
            val = df[df['Country'] == country].loc[:, year].values
            if len(val) == 0: 
                undp_data.loc[(int(year), country), nam] = np.nan
            else: undp_data.loc[(int(year), country), nam] = val[0]

In [624]:
undp_data.loc[(slice(None),'Albania'),]

,,Education Index,Employment to population ratio (% ages 15 and older),"Population, ages 65 and older (millions)","Mortality rate, infant (per 1,000 live births)","Human Development Index (HDI), female","Estimated gross national income per capita, female (2011 PPP$)",Share of seats in parliament (% held by women),"Labour force participation rate (% ages 15 and older), male","Remittances, inflows (% of GDP)",International inbound tourists (thousands),...,Domestic credit provided by financial sector (% of GDP),Forest area (% of total land area),Life expectancy Index,Adjusted net savings (% of GNI),Population using at least basic drinking-water services (%),"Tuberculosis incidence (per 100,000 people)","Mortality rate, male adult (per 1,000 people)","Population, total (millions)",Young age (0-14) dependency ratio (per 100 people ages 15-64),"Population with at least some secondary education, female (% ages 25 and older)"
year,country,,,,,,,,,,,,,,,,,,,,,
1990,Albania,0.584,NaN,0.2,34.8,NaN,NaN,NaN,74.6,NaN,NaN,...,NaN,28.8,0.797,2.3,NaN,NaN,144,3.3,53.1,59.5
1991,Albania,0.589,54.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.797,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,Albania,0.558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.797,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,Albania,0.543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.798,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,Albania,0.542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,Albania,0.551,52.3,0.2,28.5,0.603,3032,NaN,73.9,17.86,304,...,40.5,28.4,0.803,9.9,NaN,NaN,144,3.1,53.8,60.6
1996,Albania,0.558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.808,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,Albania,0.571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.813,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,Albania,0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.819,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [508]:
undp_data.loc[(1990, slice(None)), :].index

MultiIndex([(1990,                         'Afghanistan'),
            (1990,                             'Albania'),
            (1990,                             'Algeria'),
            (1990,                             'Andorra'),
            (1990,                              'Angola'),
            (1990,                 'Antigua and Barbuda'),
            (1990,                           'Argentina'),
            (1990,                             'Armenia'),
            (1990,                           'Australia'),
            (1990,                             'Austria'),
            ...
            (1990,                            ' Vanuatu'),
            (1990, ' Venezuela (Bolivarian Republic of)'),
            (1990,                           ' Viet Nam'),
            (1990,                              ' Yemen'),
            (1990,                             ' Zambia'),
            (1990,                           ' Zimbabwe'),
            (1990,                      

In [512]:
undp_data.loc[(1990, slice(None)), :].index.levels[1]

Index(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria',
       ...
       ' Vanuatu', ' Venezuela (Bolivarian Republic of)', ' Viet Nam',
       ' Yemen', ' Zambia', ' Zimbabwe', ' Andorra', ' Dominica',
       ' Saint Kitts and Nevis', ' Liechtenstein'],
      dtype='object', name='country', length=384)

In [443]:
for i,x in enumerate((undp_data.loc[(2000, ' Algeria'), :])):
    if not np.isnan(x): print(i)

44


In [628]:
undp_data.to_csv('dataset/undp_data.csv', na_rep='nan')

In [631]:
undp_data.loc[(2017, slice(None)),]

Education Index  \
year country                                              
2017 Afghanistan                                  0.408   
     Albania                                      0.758   
     Algeria                                      0.674   
     Andorra                                      0.701   
     Angola                                       0.498   
...                                                 ...   
     Venezuela (Bolivarian Republic of)             0.7   
     Viet Nam                                     0.626   
     Yemen                                        0.341   
     Zambia                                       0.572   
     Zimbabwe                                     0.568   

                                        Employment to population ratio (% ages 15 and older)  \
year country                                                                                   
2017 Afghanistan                                                                      64.9     
     Albania                                                                          48.5     
     Algeria                                                                          36.4     
     Andorra                                                                           NaN     
     Angola                                                                           72.2     
...                                                                                    ...     
     Venezuela (Bolivarian Republic of)                                               57.7     
     Viet Nam                                                                           76     
     Yemen                                                                            33.5     
     Zambia                                                                           69.8     
     Zimbabwe                                                                         79.3     

                                        Population, ages 65 and older (millions)  \
year country                                                                       
2017 Afghanistan                                                             0.9   
     Albania                                                                 0.4   
     Algeria                                                                 2.6   
     Andorra                                                                 NaN   
     Angola                                                                  0.7   
...                                                                          ...   
     Venezuela (Bolivarian Republic of)                                        2   
     Viet Nam                                                                6.7   
     Yemen                                                                   0.8   
     Zambia                                                                  0.4   
     Zimbabwe                                                                0.4   

                                        Mortality rate, infant (per 1,000 live births)  \
year country                                                                             
2017 Afghanistan                                                                  51.5   
     Albania                                                                       7.8   
     Algeria                                                                      20.6   
     Andorra                                                                       3.2   
     Angola                                                                       53.8   
...                                                                                ...   
     Venezuela (Bolivarian Republic of)                                           25.7   
     Viet Nam                                                                     16.7   
     Yemen                    

In [ ]:
data_tra

In [661]:
undp_data.columns

Index(['Education Index',
       'Employment to population ratio (% ages 15 and older)',
       'Population, ages 65 and older (millions)',
       'Mortality rate, infant (per 1,000 live births)',
       'Human Development Index (HDI), female',
       'Estimated gross national income per capita, female (2011 PPP$)',
       'Share of seats in parliament (% held by women)',
       'Labour force participation rate (% ages 15 and older), male',
       'Remittances, inflows (% of GDP)',
       'International inbound tourists (thousands)',
       'Infants lacking immunization, measles (% of one-year-olds)',
       'Foreign direct investment, net inflows (% of GDP)',
       'Employment in services (% of total employment)',
       'Labour force participation rate (% ages 15 and older)',
       'Mean years of schooling, male (years)',
       'Expected years of schooling, male (years)',
       'Mobile phone subscriptions (per 100 people)',
       'Renewable energy consumption (% of total final e

In [646]:
#Test data_trainset contains objects only of latest 10 years,
#including unmentioned in train data_train latest year 1016064
unique_years = data_train['galactic year'].unique()
print (unique_years)
print (unique_years.shape)

# print(np.sort(test['galactic year'].unique()))
# print(test['galactic year'].unique().shape)

unique_names = data_train['galaxy'].unique()
unique_names = np.sort(unique_names)
print (unique_names[:5])
print (unique_names.shape)

#Let's map all galactic years to years with increment of 1 year
# and map all galaxies to their unique integer key

#dictionary for replacement of galactic years with normal years
di = {val: ind+1990 for ind, val in enumerate(np.append(unique_years,
                                                     1016064))}
#dictionary for replacement of names with integer keys
di_names = {val: ind+1 for ind, val in enumerate(unique_names)}
data_train.replace({'galactic year':di}, inplace=True)
#test.replace({'galactic year':di}, inplace=True)

# data_train.replace({'galaxy':di_names}, inplace=True)
# test.replace({'galaxy':di_names}, inplace=True)

[ 990025  991020  992016  993012  994009  995006  996004  997002  998001
  999000 1000000 1001000 1002001 1003002 1004004 1005006 1006009 1007012
 1008016 1009020 1010025 1011030 1012036 1013042 1014049 1015056]
(26,)
['Andromeda Galaxy (M31)' 'Andromeda I' 'Andromeda II' 'Andromeda III'
 'Andromeda IX']
(181,)


In [653]:
unique_data_years = data_train['galactic year'].unique()
unique_galaxy_names = data_train['galaxy'].unique()

In [659]:
data_train_m = data_train.set_index(['galactic year', 'galaxy'])

In [668]:
# Preparing train set to compare with the UNDP dataset. I dropped some columns since I believe 
# the HDI Ranks for the year from which the original dataset was pulled might
# be different from the one in 2018
data_train_cl = data_train_m.drop(labels=['Intergalactic Development Index (IDI), Rank',
                          'Intergalactic Development Index (IDI), female, Rank',
                          'Intergalactic Development Index (IDI), male, Rank',
                          'y'
                         ],
                  axis = 1)

In [669]:
data_train_cl

existence expectancy index  \
galactic year galaxy                                                     
1990          Large Magellanic Cloud (LMC)                    0.628657   
              Camelopardalis B                                0.818082   
              Virgo I                                         0.659443   
              UGC 8651 (DDO 181)                              0.555862   
              Tucana Dwarf                                    0.991196   
...                                                                ...   
2015          Columba I                                       1.029704   
              Leo II Dwarf (Leo B, DDO 93)                    0.937869   
              Canes Venatici I Dwarf                          1.036144   
              KKs 3                                           0.939034   
              NGC 5237                                        1.032244   

                                            existence expectancy at birth  \
galactic year galaxy                                                        
1990          Large Magellanic Cloud (LMC)                      63.125200   
              Camelopardalis B                                  81.004994   
              Virgo I                                           59.570534   
              UGC 8651 (DDO 181)                                52.333293   
              Tucana Dwarf                                      81.802464   
...                                                                   ...   
2015          Columba I                                         82.832063   
              Leo II Dwarf (Leo B, DDO 93)                      75.877098   
              Canes Venatici I Dwarf                            93.540275   
              KKs 3                                             78.274427   
              NGC 5237                                          91.641356   

                                            Gross income per capita  \
galactic year galaxy                                                  
1990          Large Magellanic Cloud (LMC)             27109.234310   
              Camelopardalis B                         30166.793958   
              Virgo I                                   8441.707353   
              UGC 8651 (DDO 181)                                NaN   
              Tucana Dwarf                             81033.956906   
...                                                             ...   
2015          Columba I                                34310.471408   
              Leo II Dwarf (Leo B, DDO 93)             36899.067719   
              Canes Venatici I Dwarf                   37002.977875   
              KKs 3                                    28180.459770   
              NGC 5237                                 73109.215949   

                                            Income Index  \
galactic year galaxy                                       
1990          Large Magellanic Cloud (LMC)      0.646039   
              Camelopardalis B                  0.852246   
              Virgo I                           0.499762   
              UGC 8651 (DDO 181)                     NaN   
              Tucana Dwarf                      1.131163   
...                                                  ...   
2015          Columba I                         0.855094   
              Leo II Dwarf (Leo B, DDO 93)      0.929494   
              Canes Venatici I Dwarf            1.085245   
              KKs 3                             0.687655   
              NGC 5237                          1.207746   

                                            Expected years of education (galactic years)  \
galactic year galaxy                                                                       
1990          Large Magellanic Cloud (LMC)                                      8.240543   
              Camelopardalis B                                                 10.671823   
      

In [856]:
unique_names.size

181

In [700]:
# Inverse dictionary of col_map_abs: Contest column names -> UNDP column names
col_map_abs_inv = {v: k for k, v in col_map_abs.items()}

In [704]:
undp_order = [col_map_abs_inv.get(x) for x in data_train_cl.columns]

In [707]:
undp_clean = undp_data[undp_order]

In [708]:
undp_clean.to_csv('dataset/undp_clean.csv', na_rep='nan')

In [1013]:
# For loading 
undp_clean = pd.read_csv('dataset/undp_clean.csv', na_values='nan').set_index(['year', 'country'])

In [1014]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_train_scaled = scaler.fit_transform(data_train_cl)
undp_clean_scaled = scaler.fit_transform(undp_clean)

data_train_scaled_sliced = scaler.fit_transform(data_train_cl.loc[(2015, slice(None)),])
undp_clean_scaled_sliced = scaler.fit_transform(undp_clean.loc[(2015, slice(None)),])

In [1015]:
from sklearn.metrics.pairwise import pairwise_distances, nan_euclidean_distances

pair_dist = pairwise_distances(data_train_scaled,
                                undp_clean_scaled,
                                metric='nan_euclidean')
pair_dist_slice = pairwise_distances(data_train_scaled_sliced,
                               undp_clean_scaled_sliced,
                               metric='nan_euclidean')

In [1016]:
pair_dist_m = np.nan_to_num(pair_dist, nan=np.inf)
pair_dist_slice_m = np.nan_to_num(pair_dist_slice, nan=np.inf)

In [1017]:
np.argmin(pair_dist_m, axis = 1)

array([3886, 2671, 3886, ..., 5151,  483, 3911])

In [1018]:
print(pair_dist_m.shape)
print(pair_dist_slice_m.shape)

(3865, 5684)
(87, 196)


In [1020]:
a = np.argmin(pair_dist_slice_m, axis = 1)
print(a.shape)
a

(87,)


array([107,  36,  84, 109,   8,  34, 170,  86,  82,   2,  30,  73, 139,
       152, 146, 174, 125,  66, 137,  24,  76, 105,  12,  64, 189, 144,
        20, 168,  80,  47,  99,   0,  88,  57, 135,  62, 118, 184,  93,
        13,  22,  28,  90, 122,  69, 115, 149, 161, 154,  26, 181, 141,
        40, 179,  14,  45,  97, 131, 176, 127,  10,  18, 129,  16,  32,
        60, 112, 103,  95, 157, 164, 191,  35, 194, 166,  43,  71, 172,
         6, 120, 159,  38,  50,  52,  55, 133,  31])

In [959]:
data_train_cl.loc[(2015, slice(None)),].index[9]

(2015, 'Tucana III')

In [960]:
undp_clean.loc[(2015, slice(None)),].index[2]

(2015, 'Algeria')

In [1047]:
# Compare averages for each galaxy across all years with the same for countries
data_train_aver = []
for galaxy in unique_names:
    data_train_aver.append(data_train_cl.loc[(slice(None), galaxy),].mean(axis = 0).values)

In [1048]:
undp_clean_aver = []
for country in full_list:
    undp_clean_aver.append(undp_clean.loc[(slice(None), country),].mean(axis = 0).values)

In [1100]:
scaler.fit(data_train_aver + undp_clean_aver)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [1118]:
np.array(data_train_aver + undp_clean_aver).shape

(377, 74)

In [1101]:
pair_dist_aver = pairwise_distances(scaler.transform(data_train_aver),
                                    scaler.transform(undp_clean_aver),
                                    metric='nan_euclidean')

In [1102]:
pair_dist_aver.shape

(181, 196)

In [1103]:
a = np.argmin(pair_dist_aver, axis = 1)
print(len(a))
print(len(set(a)))
print(a)

181
95
[ 90  95 165  55  61  86  98 104  73 136 109 126  30 134   3  79   3 105
  99  55  72  78  55  60 120 117 111  38  62  22 126 134 147  34  55 169
  49  78  64   3  82  66   3  77  56   3  55   4  55 165  27   0   3  72
 167 131 174   3 193   3  64 192 134  11 125  36   3  20 147 126 126 185
   3  30  30  64 169   4 194 126 134   3 126 170 195 165 157  55 134 129
 122 143 140  78 124 134  19 138  57   3 131 125 169  47   3  49  55 158
 169 161 194  35 183  64  94 126 190  33  48 169  64 153 150 187 194 187
  79 151   3  92  53 151 187  35 125   3  29   3   3  64  66  10  33   3
 126  55  31   4  11 118 121  30  23   3 170 103 118   3  30  30  64 145
 169   4 175 136 146 177 194 181 134 168   3  36 163   3   8  24  32  12
 108]


In [1113]:
print (unique_names[12])
print (full_list[126])

Andromeda XVII
New Zealand


In [1087]:
data_train_cl.loc[(slice(None), 'Andromeda XVII'),]

,,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),...,Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII)
galactic year,galaxy,,,,,,,,,,,,,,,,,,,,,
1990,Andromeda XVII,0.570597,56.224164,21346.294277,0.702896,6.571980,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22.358389,0.388783,NaN,NaN
1991,Andromeda XVII,0.609940,52.122608,18507.252898,0.561551,7.217000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,Andromeda XVII,0.608488,57.619002,20909.967987,0.645434,5.646878,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,Andromeda XVII,0.542447,56.738724,17080.548819,0.654083,6.941214,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,Andromeda XVII,0.592923,54.876945,26845.459395,0.654931,8.808054,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,Andromeda XVII,0.570682,56.664377,23712.331408,0.604377,7.245093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.999705,6.353315,NaN,NaN
1996,Andromeda XVII,0.577999,62.496087,12836.420220,0.631138,9.089118,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,Andromeda XVII,0.644313,62.096380,21403.937280,0.670473,10.303041,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,Andromeda XVII,0.681252,59.106292,14997.051816,0.660937,10.327224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1114]:
undp_clean.loc[(slice(None), 'New Zealand'),]

,,Life expectancy Index,Life expectancy at birth,Gross national income (GNI) per capita (2011 PPP$),Income Index,Expected years of schooling (years),Mean years of schooling (years),Human Development Index (HDI),Education Index,Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),...,Mobile phone subscriptions (per 100 people),"Internet users, total (% of population)",Current health expenditure (% of GDP),"Human Development Index (HDI), female","Human Development Index (HDI), male",Gender Development Index (GDI),Adjusted net savings (% of GNI),"HIV prevalence, adult (% ages 15-49), total",Private capital flows (% of GDP),Gender Inequality Index (GII)
year,country,,,,,,,,,,,,,,,,,,,,,
1990,New Zealand,0.852,75.4,23099.0,0.822,14.5,11.5,0.820,0.787,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,NaN,NaN
1991,New Zealand,0.857,75.7,21231.0,0.809,14.8,11.6,0.821,0.797,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,New Zealand,0.863,76.1,21294.0,0.810,15.2,11.6,0.826,0.807,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,New Zealand,0.868,76.4,22228.0,0.816,16.0,11.6,0.838,0.829,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,New Zealand,0.873,76.7,23146.0,0.822,16.6,11.6,0.847,0.846,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,New Zealand,0.877,77.0,24019.0,0.828,16.9,11.6,0.853,0.854,NaN,NaN,...,NaN,NaN,NaN,0.838,0.863,0.970,NaN,0.1,NaN,0.222
1996,New Zealand,0.881,77.2,24213.0,0.829,17.0,11.6,0.856,0.858,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,New Zealand,0.885,77.5,24725.0,0.832,17.3,11.6,0.861,0.867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,New Zealand,0.889,77.8,25064.0,0.834,17.4,11.6,0.864,0.869,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
